In [26]:
import os
from dotenv import load_dotenv
import requests

env_path = ".env"
load_dotenv(dotenv_path=env_path)

from services import zoom_service
from services import chatgpt_service
from utils.ai_transcript_chunking import generate_overlapping_chunk
from utils.action_item_parsing import action_item_processing

In [2]:
meeting_id = 85317636024

In [3]:
current_dir = os.getcwd()
os.chdir("services")
zoom_transcript_file = zoom_service.get_meeting_transcript(meeting_id)
with open(zoom_transcript_file) as f:
    zoom_transcript_string = f.read()

In [4]:
parsed_2d_list = generate_overlapping_chunk(zoom_transcript_string)

In [6]:
overlapped = []
for chunk in parsed_2d_list:
    string = '\n'.join(chunk)
    action_items_arr = None
    while True:
        action_items_arr = chatgpt_service.extract_action_item(string)
        if action_items_arr is not None:
            break
    overlapped.append(action_items_arr)

ChatGPT: extracting action items
ChatGPT: extracting action items
ChatGPT: extracting action items


In [20]:
combined_action_string = " ".join([lst for lsts in overlapped for lst in lsts])

In [24]:
API_KEY = os.environ['OPENAI_API_KEY']

In [29]:
def combine_action_item(action_items_lists):
    """
    Combines a list of action item lists into a single string and uses a GPT-3.5 model to generate
    distinct action items. The model response is parsed and returned as a list of strings.

    Args:
        action_items_lists (list): A list of action item lists, where each inner list contains strings
                                   representing action items.

    Returns:
        list: A list of distinct action items generated by the GPT-3.5 model.
    """
    
    combined_action_string = " ".join([lst for lsts in overlapped for lst in lsts])
    
    print("ChatGPT: combining action items")
    prompt = "Find all the distinct action items using this format: \"{WHO}: {ACTION ITEM}\""
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You need to drop duplicated action items and combine them in the given format."},
            {"role": "assistant", "content": combined_action_string},
            {"role": "user", "content": prompt},
        ],
        "temperature": 0
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        action_items_string = response.json()['choices'][0]['message']['content']
        return [a for a in action_items_string.split('\n') if len(a) > 0]
    else:
        raise ValueError("Failure to get GPT response")

In [30]:
combine_action_item(overlapped)

ChatGPT: combining action items


['DAZHI PENG: Read the documentation with the beautiful soup documentation.',
 'DAZHI PENG: Help Yooni with Python installation issues.',
 'DAZHI PENG: Send me the link to your Github branch.',
 'DAZHI PENG: Schedule a meeting with Jeffrey Feng for recordings of the previous project.',
 'DAZHI PENG: Send an update on new team members for the project in a week and try to find additional team members.',
 'Jeffrey Feng: Research and try using the selenium library to scrape dynamic websites.',
 'Jeffrey Feng: Help Yooni with Python installation issues.',
 'Jeffrey Feng: Schedule a meeting with DAZHI PENG and try to add Yooni Choi to the team.',
 'Yooni Choi: Update Python environment.',
 'Yooni Choi: Troubleshoot Python installation issues.',
 'Yooni Choi: Finish the Beautiful Soup part - Report progress to DAZHI PENG by tomorrow.']